In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path = 'C:/Users/wjssm/Desktop/kaggle/Elo_Merchant_Category_Recommendation/data/'

train = pd.read_csv(path + 'train.csv') #key : card_id
test = pd.read_csv(path + 'test.csv') #key : card_id
#history = pd.read_csv(path+'historical_transactions.csv') #key : card_id, merchant_id
#new_history = pd.read_csv(path+'new_merchant_transactions.csv') #key : card_id, merchant_id
merchant = pd.read_csv(path+'merchants.csv') #key : merchant_id

In [3]:
history = pd.read_pickle(path + 'sorted_history.pkl')
history.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
19095896,Y,C_ID_00007093c1,244,N,2,C,705,M_ID_69423b34e4,-12,0.037176,2017-02-14 14:00:43,3.0,2,33
19095775,Y,C_ID_00007093c1,244,N,1,B,705,M_ID_a33355a1b7,-12,-0.467415,2017-02-14 15:47:45,3.0,2,33
19095845,Y,C_ID_00007093c1,244,N,1,B,307,M_ID_9400cf2342,-12,-0.572600,2017-02-16 15:37:58,3.0,2,19
19095866,Y,C_ID_00007093c1,244,N,3,C,705,M_ID_a33355a1b7,-12,-0.331575,2017-02-20 12:19:01,3.0,2,33
19095808,Y,C_ID_00007093c1,244,N,1,B,307,M_ID_9400cf2342,-11,-0.483944,2017-03-03 00:24:15,3.0,2,19


In [3]:
train['year'] = pd.DatetimeIndex(train['first_active_month']).year
train['month'] = pd.DatetimeIndex(train['first_active_month']).month

test['year'] = pd.DatetimeIndex(test['first_active_month']).year
test['month'] = pd.DatetimeIndex(test['first_active_month']).month

In [4]:
#test data에서의 NA어떻게 할지...
test.loc[test.first_active_month.isnull(),:]

,first_active_month,card_id,feature_1,feature_2,feature_3,year,month
11578,NaN,C_ID_c27b4f80f7,5,2,1,NaN,NaN


In [9]:
test.loc[test.card_id == 'C_ID_c27b4f80f7', 'year'] = 2017 #최빈값
test.loc[test.card_id == 'C_ID_c27b4f80f7', 'month'] = 3 #가장 오래된 거래가 2017.03

In [7]:
history.loc[history.card_id == 'C_ID_c27b4f80f7', :]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
25672893,N,C_ID_c27b4f80f7,-1,Y,10,C,511,M_ID_b9dcf28cb9,-11,-0.056289,2017-03-09 01:34:22,NaN,-1,7
25672920,N,C_ID_c27b4f80f7,-1,Y,10,C,511,M_ID_b9dcf28cb9,-11,-0.407564,2017-03-09 01:46:07,NaN,-1,7
25672896,N,C_ID_c27b4f80f7,-1,Y,10,C,511,M_ID_b9dcf28cb9,-11,-0.056289,2017-03-10 20:41:16,NaN,-1,7
25672907,N,C_ID_c27b4f80f7,-1,Y,10,C,511,M_ID_b9dcf28cb9,-11,-0.056289,2017-03-11 02:46:55,NaN,-1,7
25672941,N,C_ID_c27b4f80f7,-1,Y,10,C,511,M_ID_b9dcf28cb9,-11,-0.407564,2017-03-11 02:57:50,NaN,-1,7
25672938,Y,C_ID_c27b4f80f7,-1,Y,1,B,511,M_ID_b9dcf28cb9,-11,-0.745405,2017-03-25 00:36:49,NaN,-1,7
25672934,Y,C_ID_c27b4f80f7,-1,Y,1,B,111,M_ID_21f1585175,-9,-0.745405,2017-05-12 23:15:44,NaN,-1,36
25672899,N,C_ID_c27b4f80f7,-1,Y,6,C,111,M_ID_21f1585175,-8,-0.351109,2017-06-12 11:35:10,NaN,-1,36
25672906,N,C_ID_c27b4f80f7,-1,Y,6,C,111,M_ID_21f1585175,-8,-0.351109,2017-06-15 11:47:24,NaN,-1,36
25672923,N,C_ID_c27b4f80f7,-1,Y,1,B,111,M_ID_21f1585175,-8,-0.680941,2017-06-15 12:29:25,NaN,-1,36


In [19]:
#merged : history + new_history -> sorted
transaction = pd.concat([history, new_history]).sort_values(['card_id','purchase_date'])
transaction.index = list(range(31075392))
#transaction.to_pickle('merged_transaction')
transaction.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
19095896,Y,C_ID_00007093c1,244,N,2,C,705,M_ID_69423b34e4,-12,0.037176,2017-02-14 14:00:43,3.0,2,33
19095775,Y,C_ID_00007093c1,244,N,1,B,705,M_ID_a33355a1b7,-12,-0.467415,2017-02-14 15:47:45,3.0,2,33
19095845,Y,C_ID_00007093c1,244,N,1,B,307,M_ID_9400cf2342,-12,-0.572600,2017-02-16 15:37:58,3.0,2,19
19095866,Y,C_ID_00007093c1,244,N,3,C,705,M_ID_a33355a1b7,-12,-0.331575,2017-02-20 12:19:01,3.0,2,33
19095808,Y,C_ID_00007093c1,244,N,1,B,307,M_ID_9400cf2342,-11,-0.483944,2017-03-03 00:24:15,3.0,2,19


### 1. authorized_flag

# authorized/not_authorized :  카드 아이디별 승인/비승인 건수

In [79]:
card_authorized = transaction.groupby(['card_id','authorized_flag'])['authorized_flag'].count()
card_authorized = pd.DataFrame(card_authorized).unstack('authorized_flag')
card_authorized.head()

authorized_flag       
authorized_flag               N      Y
card_id                               
C_ID_00007093c1            35.0  116.0
C_ID_0001238066             3.0  146.0
C_ID_0001506ef0             4.0   64.0
C_ID_0001793786            27.0  220.0
C_ID_000183fdda             7.0  148.0

In [88]:
transaction['authorized_flag'].isnull().sum()

0

In [84]:
authorize = pd.concat([pd.Series(card_authorized[("authorized_flag", "Y")].values), pd.Series(card_authorized[("authorized_flag", "N")].values) ], axis = 1)
authorize.index = transaction.card_id.unique()
authorize.columns = ['authorized','not_authorized']
authorize.head()

,authorized,not_authorized
C_ID_00007093c1,116.0,35.0
C_ID_0001238066,146.0,3.0
C_ID_0001506ef0,64.0,4.0
C_ID_0001793786,220.0,27.0
C_ID_000183fdda,148.0,7.0


In [91]:
authorize.fillna(0,inplace=True)

In [114]:
authorize['authorized_rate'] = authorize['not_authorized'] / authorize['authorized']
authorize.head()

,authorized,not_authorized,authorized_rate
C_ID_00007093c1,116.0,35.0,0.301724
C_ID_0001238066,146.0,3.0,0.020548
C_ID_0001506ef0,64.0,4.0,0.062500
C_ID_0001793786,220.0,27.0,0.122727
C_ID_000183fdda,148.0,7.0,0.047297


In [113]:
print('전체 비승인/승인 비율 : ', sum(transaction.authorized_flag == 'N') / sum(transaction.authorized_flag == 'Y') )

전체 승인/비승인 비율 :  0.0881317470539314


### history - installments outlier

In [94]:
transaction['installments'].value_counts()

 0      16333991
 1      12513700
 2        721145
 3        582957
-1        234081
 4        194340
 6        143023
 10       127726
 5        125386
 12        57914
 8         22029
 7         11769
 9          6250
 11          891
 999         190
Name: installments, dtype: int64

### 2. installments : 할부

#### 999

In [95]:
#category3가 B == installments 1
print('category_3 = B인 개수 : ', transaction.loc[transaction.category_3 == 'B',:].shape[0])
print('installments = 1인 개수 : ', transaction.loc[transaction.installments == 1,:].shape[0])

category_3 = B인 개수 :  12513700
installments = 1인 개수 :  12513700


In [96]:
transaction.loc[transaction.installments != 999, 'category_3'].value_counts()

A    16333991
B    12513700
C     1993430
Name: category_3, dtype: int64

In [97]:
#installments 999 -> category3 다 C
transaction.loc[transaction.installments == 999, 'category_3'].value_counts()

C    190
Name: category_3, dtype: int64

In [98]:
##역은 성립 x
transaction.loc[transaction.category_3 == 'C', 'installments'].value_counts()

2      721145
3      582957
4      194340
6      143023
10     127726
5      125386
12      57914
8       22029
7       11769
9        6250
11        891
999       190
Name: installments, dtype: int64

In [127]:
transaction.loc[transaction.installments == -1]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
229,Y,C_ID_0001238066,187,N,-1,NaN,367,M_ID_3f35ef9a50,-1,-0.605012,2018-01-06 12:14:16,1.0,15,16
235,Y,C_ID_0001238066,-1,N,-1,NaN,333,M_ID_40d7b120ab,-1,0.244842,2018-01-18 13:58:35,NaN,-1,10
240,Y,C_ID_0001238066,160,N,-1,NaN,451,M_ID_970d4d00da,-1,-0.161475,2018-01-23 11:44:30,5.0,21,7
279,Y,C_ID_0001238066,69,N,-1,NaN,87,M_ID_b34bebaf3c,1,-0.152008,2018-03-16 18:49:21,1.0,9,27
697,Y,C_ID_000183fdda,87,N,-1,NaN,369,M_ID_67894eed56,-2,-0.506634,2017-12-28 16:56:20,3.0,11,27
699,Y,C_ID_000183fdda,161,N,-1,NaN,68,M_ID_f4e377b35d,-2,-0.091752,2017-12-29 11:31:47,3.0,3,27
716,Y,C_ID_000183fdda,161,N,-1,NaN,68,M_ID_f4e377b35d,-1,-0.460833,2018-01-12 10:33:26,3.0,3,27
717,Y,C_ID_000183fdda,161,N,-1,NaN,422,M_ID_4b99ea1c74,-1,-0.536537,2018-01-12 12:18:39,3.0,3,27
769,Y,C_ID_000183fdda,161,N,-1,NaN,489,M_ID_3bf000781a,2,-0.107680,2018-04-30 14:59:53,3.0,3,16
878,Y,C_ID_0002709b5a,103,N,-1,NaN,60,M_ID_5bb84e5819,-4,-0.645779,2017-10-28 00:00:00,2.0,18,13


In [131]:
transaction.loc[(transaction.merchant_id == 'M_ID_40d7b120ab') & (transaction.card_id == 'C_ID_0001238066')]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
235,Y,C_ID_0001238066,-1,N,-1,NaN,333,M_ID_40d7b120ab,-1,0.244842,2018-01-18 13:58:35,NaN,-1,10


In [119]:
#installments 999인 data
installments_999 = transaction.loc[transaction.installments == 999]
installments_999.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
338010,N,C_ID_02bd1c4210,322,N,999,C,369,M_ID_44d94f35bb,-12,4.963167,2017-02-04 17:03:01,3.0,11,27
632347,N,C_ID_05240c2353,289,N,999,C,60,M_ID_dc44de5cf9,-2,0.697140,2017-08-30 11:35:02,5.0,5,13
858170,N,C_ID_06fca19c8e,331,N,999,C,222,M_ID_629b908f04,-6,1.036739,2017-08-10 13:26:40,1.0,16,21
1128016,N,C_ID_094bb1b704,69,N,999,C,412,M_ID_dfc0be4604,-5,1.507069,2017-03-17 18:52:58,1.0,9,18
1195721,N,C_ID_09dcde8f3c,137,N,999,C,891,M_ID_d0d69f7b22,-8,1.431335,2017-05-20 13:34:50,3.0,19,29


In [126]:
installments_999.loc[installments_999.authorized_flag == 'Y', :]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
3075047,Y,C_ID_1947a910eb,174,N,999,C,840,M_ID_fcb061b2bf,-2,2.709190,2017-12-17 22:36:08,1.0,9,20
5642431,Y,C_ID_2e39f9fd94,340,N,999,C,840,M_ID_00f803a717,-4,0.754241,2017-10-28 13:46:13,1.0,9,20
9404025,Y,C_ID_4d3dbbbb1f,159,N,999,C,840,M_ID_e243972510,-2,0.755743,2017-12-28 19:27:12,1.0,9,20
12566258,Y,C_ID_6715eccea8,174,N,999,C,840,M_ID_fcb061b2bf,-2,0.491983,2017-11-23 23:24:39,1.0,9,20
13934716,Y,C_ID_726ae3c2bc,174,N,999,C,840,M_ID_fcb061b2bf,2,1.032186,2017-12-23 23:57:41,1.0,9,20
15926438,Y,C_ID_82e68c6826,174,N,999,C,840,M_ID_fcb061b2bf,-5,1.806097,2017-09-21 16:37:51,1.0,9,20
16482622,Y,C_ID_875f80a30c,340,N,999,C,840,M_ID_995feeec61,-3,4.435736,2017-09-20 18:21:42,1.0,9,20
16958974,Y,C_ID_8b3e96516a,174,N,999,C,840,M_ID_fcb061b2bf,1,-0.027138,2017-04-24 20:04:23,1.0,9,20


In [125]:
installments_999.authorized_flag.value_counts()

N    182
Y      8
Name: authorized_flag, dtype: int64

In [120]:
installments_999.shape

(190, 14)

In [121]:
print('card_id unique 개수 : ', len(installments_999.card_id.unique()))
print('merchant_id unique 개수 : ', len(installments_999.merchant_id.unique()))

card_id unique 개수 :  180
merchant_id unique 개수 :  171


In [122]:
#승인이 여러번 N 된 경우 -> 같은 카드 아이디일 경우, 같은 merchant_id
installments_999.card_id.value_counts().head(5)

C_ID_1fa5e84ac3    6
C_ID_f8beeb608f    3
C_ID_2e39f9fd94    2
C_ID_76859ed625    2
C_ID_105ae2e0d1    2
Name: card_id, dtype: int64

In [123]:
#같은 날 비슷한 시간대에 일어남
installments_999.loc[installments_999.card_id == 'C_ID_1fa5e84ac3', :]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
3870739,N,C_ID_1fa5e84ac3,-1,Y,999,C,690,M_ID_3111c6df35,-3,1.619738,2017-11-27 11:07:18,NaN,-1,1
3870740,N,C_ID_1fa5e84ac3,-1,Y,999,C,690,M_ID_3111c6df35,-3,1.619738,2017-11-27 11:07:19,NaN,-1,1
3870741,N,C_ID_1fa5e84ac3,-1,Y,999,C,690,M_ID_3111c6df35,-3,1.619738,2017-11-27 11:58:12,NaN,-1,1
3870742,N,C_ID_1fa5e84ac3,-1,Y,999,C,690,M_ID_3111c6df35,-3,1.619738,2017-11-27 11:58:13,NaN,-1,1
3870743,N,C_ID_1fa5e84ac3,-1,Y,999,C,690,M_ID_3111c6df35,-3,1.619738,2017-11-27 12:50:38,NaN,-1,1
3870744,N,C_ID_1fa5e84ac3,-1,Y,999,C,690,M_ID_3111c6df35,-3,1.619738,2017-11-27 13:41:26,NaN,-1,1


In [110]:
#같은 날 
transaction.loc[(transaction.card_id == 'C_ID_1fa5e84ac3') & (transaction.merchant_id =='M_ID_3111c6df35')]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
3870739,N,C_ID_1fa5e84ac3,-1,Y,999,C,690,M_ID_3111c6df35,-3,1.619738,2017-11-27 11:07:18,NaN,-1,1
3870740,N,C_ID_1fa5e84ac3,-1,Y,999,C,690,M_ID_3111c6df35,-3,1.619738,2017-11-27 11:07:19,NaN,-1,1
3870741,N,C_ID_1fa5e84ac3,-1,Y,999,C,690,M_ID_3111c6df35,-3,1.619738,2017-11-27 11:58:12,NaN,-1,1
3870742,N,C_ID_1fa5e84ac3,-1,Y,999,C,690,M_ID_3111c6df35,-3,1.619738,2017-11-27 11:58:13,NaN,-1,1
3870743,N,C_ID_1fa5e84ac3,-1,Y,999,C,690,M_ID_3111c6df35,-3,1.619738,2017-11-27 12:50:38,NaN,-1,1
3870744,N,C_ID_1fa5e84ac3,-1,Y,999,C,690,M_ID_3111c6df35,-3,1.619738,2017-11-27 13:41:26,NaN,-1,1


In [70]:
installments_999.loc[installments_999.card_id == 'C_ID_f8beeb608f', :]

,card_id,merchant_id
27441288,C_ID_f8beeb608f,M_ID_0bef30db30
27441292,C_ID_f8beeb608f,M_ID_0bef30db30
27441309,C_ID_f8beeb608f,M_ID_0bef30db30


In [133]:
#같은 날 여러번 시도 끝에 승인
transaction.loc[(transaction.card_id == 'C_ID_f8beeb608f') & (transaction.merchant_id =='M_ID_0bef30db30')]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
30175184,N,C_ID_f8beeb608f,158,N,7,C,246,M_ID_0bef30db30,0,15.556857,2018-02-09 13:43:22,1.0,15,7
30175185,N,C_ID_f8beeb608f,158,N,6,C,246,M_ID_0bef30db30,0,13.227748,2018-02-09 13:44:23,1.0,15,7
30175186,N,C_ID_f8beeb608f,158,N,999,C,246,M_ID_0bef30db30,0,45.835279,2018-02-09 14:02:02,1.0,15,7
30175187,N,C_ID_f8beeb608f,158,N,999,C,246,M_ID_0bef30db30,0,45.835279,2018-02-09 14:03:28,1.0,15,7
30175188,N,C_ID_f8beeb608f,158,N,999,C,246,M_ID_0bef30db30,0,45.835279,2018-02-09 14:08:48,1.0,15,7
30175189,Y,C_ID_f8beeb608f,158,N,10,C,246,M_ID_0bef30db30,0,22.544185,2018-02-09 14:10:28,1.0,15,7


In [71]:
installments_999.loc[installments_999.card_id == 'C_ID_2e39f9fd94', :]

,card_id,merchant_id
13456410,C_ID_2e39f9fd94,M_ID_00f803a717
13456426,C_ID_2e39f9fd94,M_ID_00f803a717


In [135]:
#같은 날 승인 후, 다시 ?
transaction.loc[(transaction.card_id == 'C_ID_2e39f9fd94') & (transaction.merchant_id =='M_ID_00f803a717')]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
5642431,Y,C_ID_2e39f9fd94,340,N,999,C,840,M_ID_00f803a717,-4,0.754241,2017-10-28 13:46:13,1.0,9,20
5642432,N,C_ID_2e39f9fd94,340,N,999,C,840,M_ID_00f803a717,-4,0.754241,2017-10-28 17:54:47,1.0,9,20


In [72]:
installments_999.loc[installments_999.card_id == 'C_ID_105ae2e0d1', :]

,card_id,merchant_id
4024235,C_ID_105ae2e0d1,M_ID_43d0f0f4dc
4024281,C_ID_105ae2e0d1,M_ID_43d0f0f4dc


In [136]:
#같은 날 여러번 시도 끝에 승인받지 못 하고 다른 날 승인
transaction.loc[(transaction.card_id == 'C_ID_105ae2e0d1') & (transaction.merchant_id =='M_ID_43d0f0f4dc')]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
1992686,N,C_ID_105ae2e0d1,125,N,999,C,792,M_ID_43d0f0f4dc,0,3.492071,2018-02-07 16:59:36,1.0,16,9
1992687,N,C_ID_105ae2e0d1,125,N,999,C,792,M_ID_43d0f0f4dc,0,3.492071,2018-02-07 17:00:33,1.0,16,9
1992692,Y,C_ID_105ae2e0d1,125,N,12,C,792,M_ID_43d0f0f4dc,0,3.492071,2018-02-09 16:51:48,1.0,16,9


In [73]:
installments_999.loc[installments_999.card_id == 'C_ID_76859ed625', :]

,card_id,merchant_id
8010607,C_ID_76859ed625,M_ID_c9714065ed
8010833,C_ID_76859ed625,M_ID_c9714065ed


In [137]:
#같은 날 여러번 시도 끝에 승인
transaction.loc[(transaction.card_id == 'C_ID_76859ed625') & (transaction.merchant_id =='M_ID_c9714065ed')]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
14434730,N,C_ID_76859ed625,69,N,999,C,34,M_ID_c9714065ed,-8,1.657334,2017-06-03 10:35:51,1.0,9,38
14434731,N,C_ID_76859ed625,69,N,999,C,34,M_ID_c9714065ed,-8,1.657334,2017-06-03 10:36:59,1.0,9,38
14434732,Y,C_ID_76859ed625,69,N,12,C,34,M_ID_c9714065ed,-8,1.657334,2017-06-03 10:38:12,1.0,9,38


# merchant_n : 같은 가게를 다른 날 몇 번 방문할까


In [ ]:
transaction['purchase_date']

In [43]:
history.loc[history.authorized_flag == 'Y', :].groupby(['card_id', 'merchant_id'])['merchant_id'].count()

card_id          merchant_id    
C_ID_00007093c1  M_ID_0379adb435     4
                 M_ID_06a8d84366     1
                 M_ID_0dabc5a70c     2
                 M_ID_109e9d2b1d     2
                 M_ID_15446e939c     1
                 M_ID_1b440d74fb     1
                 M_ID_24e067cb2b     1
                 M_ID_2efbd71a00     1
                 M_ID_3c6556f2dd     1
                 M_ID_490fc654fb     1
                 M_ID_5357547c3e     4
                 M_ID_57df19bf28    24
                 M_ID_584ad6cd63     3
                 M_ID_58fa2d2ab7     1
                 M_ID_61f1337e5d     1
                 M_ID_69423b34e4    10
                 M_ID_6a1f75f661     3
                 M_ID_8261f7a734     2
                 M_ID_84e60960a3     3
                 M_ID_9400cf2342    31
                 M_ID_a33355a1b7     4
                 M_ID_a614b6d5b0     1
                 M_ID_bc3f2e986e     3
                 M_ID_c6ca6282e6     1
                 M_ID_d96a11b2d

In [ ]:
merchant_authorized = 

In [ ]:
#같은 상품을 몇 번 사는지 생각해 볼 수 있지 않을까

In [31]:
history.loc[(history.merchant_id=='M_ID_0155450c84') & (history.card_id=='C_ID_8b04d4734e')]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
227606,N,C_ID_8b04d4734e,210,N,999,C,806,M_ID_0155450c84,-11,6.237415,2017-03-28 19:40:50,1.0,16,9
227803,Y,C_ID_8b04d4734e,210,N,12,C,806,M_ID_0155450c84,-11,6.237866,2017-03-28 19:41:53,1.0,16,9


In [12]:
sort_index = history.sort_values(['card_id','merchant_id','purchase_date'], ascending=True).index.tolist()

In [13]:
sort_index = np.array(sort_index)

In [15]:
history.sort_values(['card_id','merchant_id','purchase_date'], ascending = True).loc[history.installments==999, :]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
10482837,N,C_ID_02bd1c4210,322,N,999,C,369,M_ID_44d94f35bb,-12,4.963167,2017-02-04 17:03:01,3.0,11,27
28996126,N,C_ID_05240c2353,289,N,999,C,60,M_ID_dc44de5cf9,-2,0.697140,2017-08-30 11:35:02,5.0,5,13
21241132,N,C_ID_06fca19c8e,331,N,999,C,222,M_ID_629b908f04,-6,1.036739,2017-08-10 13:26:40,1.0,16,21
10410638,N,C_ID_094bb1b704,69,N,999,C,412,M_ID_dfc0be4604,-5,1.507069,2017-03-17 18:52:58,1.0,9,18
8549841,N,C_ID_09dcde8f3c,137,N,999,C,891,M_ID_d0d69f7b22,-8,1.431335,2017-05-20 13:34:50,3.0,19,29
28258830,N,C_ID_0a0fcf2fc3,137,N,999,C,544,M_ID_a5e5e3e2cc,-5,-0.623690,2017-09-27 17:44:56,3.0,19,29
2063505,N,C_ID_0a3724504a,279,N,999,C,667,M_ID_45bfdc72e8,-4,-0.259147,2017-10-13 18:46:21,1.0,9,21
21572523,N,C_ID_0ba083756a,174,N,999,C,670,M_ID_923a7d9981,-1,7.367409,2018-01-13 15:29:43,1.0,9,32
27078432,N,C_ID_0e00a3c5e2,333,N,999,C,705,M_ID_77e2942cd8,-1,-0.480984,2018-01-12 18:51:35,1.0,9,33
8382138,N,C_ID_0f1f3c5b68,69,N,999,C,422,M_ID_de738c952d,-7,-0.326180,2017-07-21 17:13:54,1.0,9,27


In [16]:
#installments 999인 index
install_999 = history.loc[history.installments==999, :].index.tolist()

In [20]:
install_999[0]

227606

In [26]:
install_999_p = []

In [18]:
sort_index

array([19095871, 19095846, 19095840, ..., 22821171, 22821184, 22821209])

In [27]:
for a in install_999 :
    install_999_p.append(sort_index[np.where(sort_index == a)[0][0]+1])

In [33]:
len(install_999) == len(install_999_p)

True

In [36]:
index_merge = list(zip(install_999, install_999_p))

In [38]:
index_merge[0]

(227606, 227803)

In [45]:
tf_sum = []

for i_m in index_merge :
    tf_sum.append(sum(history.iloc[i_m[0],:] == history.iloc[i_m[1],:]))

In [56]:
tf_sum = np.array(tf_sum)
tf_sum_where = np.where(tf_sum < 10)
tf_sum_where[0]

array([  9,  31,  34,  36,  42,  49,  55,  58,  67,  74,  85,  92,  94,
       100, 123, 129, 130, 133, 138, 139, 142, 152, 155, 170, 174, 182,
       186], dtype=int64)

In [52]:
#999떴는데 바로 재결제 안 한 값들
not_re_purchase = pd.DataFrame(columns=history.columns)
not_re_purchase

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id


In [ ]:
not_re_purchase = pd.concat([not_re_purchase, ])

In [57]:
for where in tf_sum_where[0] :
    not_re_purchase = pd.concat([not_re_purchase,history.iloc[[index_merge[where][0], index_merge[where][1]], :]])

In [58]:
not_re_purchase

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
2063505,N,C_ID_0a3724504a,279,N,999,C,667,M_ID_45bfdc72e8,-4,-0.259147,2017-10-13 18:46:21,1.0,9,21
2063635,Y,C_ID_0a3724504a,69,N,1,B,278,M_ID_45c70efb27,-9,-0.656749,2017-05-18 20:13:52,1.0,9,37
8306029,N,C_ID_eb063adeca,280,N,999,C,333,M_ID_c7e404e68b,-10,3.761046,2017-04-03 11:13:03,3.0,1,10
8306019,Y,C_ID_eb063adeca,277,N,1,B,683,M_ID_c9000570e6,-5,-0.638717,2017-09-28 13:03:28,4.0,13,34
8549841,N,C_ID_09dcde8f3c,137,N,999,C,891,M_ID_d0d69f7b22,-8,1.431335,2017-05-20 13:34:50,3.0,19,29
8549796,Y,C_ID_09dcde8f3c,147,N,1,B,690,M_ID_e12102d71d,-6,-0.697380,2017-07-08 12:29:53,3.0,19,1
8885782,N,C_ID_b1fe6776d4,-1,N,999,C,373,M_ID_7990ce34c9,-7,-0.506484,2017-07-07 13:30:26,NaN,-1,18
8885601,Y,C_ID_b1fe6776d4,-1,N,1,B,278,M_ID_7b3b569b07,-8,-0.700566,2017-06-17 13:24:13,NaN,-1,37
9958645,Y,C_ID_875f80a30c,340,N,999,C,840,M_ID_995feeec61,-3,4.435736,2017-09-20 18:21:42,1.0,9,20
9958734,Y,C_ID_875f80a30c,340,N,1,B,504,M_ID_9960e8263f,-2,-0.643225,2017-10-01 03:30:28,1.0,9,25


In [74]:
history.loc[(history.card_id=='C_ID_0a3724504a') & (history.merchant_id == 'M_ID_45bfdc72e8'), :].sort_values(['purchase_date'],ascending=True)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
2063667,Y,C_ID_0a3724504a,279,N,6,C,667,M_ID_45bfdc72e8,-4,0.167005,2017-10-13 11:49:52,1.0,9,21
2063505,N,C_ID_0a3724504a,279,N,999,C,667,M_ID_45bfdc72e8,-4,-0.259147,2017-10-13 18:46:21,1.0,9,21


In [75]:
history.loc[(history.card_id=='C_ID_eb063adeca') & (history.merchant_id == 'M_ID_c7e404e68b'), :].sort_values(['purchase_date'],ascending=True)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
8306029,N,C_ID_eb063adeca,280,N,999,C,333,M_ID_c7e404e68b,-10,3.761046,2017-04-03 11:13:03,3.0,1,10


In [144]:
history.loc[history.card_id=='C_ID_7670e569a0', :].sort_values(['merchant_id','purchase_date']).loc[history.installments!=1,
    history.columns.str.endswith('_id') | history.columns.str.startswith('purchase')|history.columns.str.endswith('installments')
     | history.columns.str.endswith('flag')]

,authorized_flag,card_id,city_id,installments,merchant_category_id,merchant_id,purchase_amount,purchase_date,state_id,subsector_id
466519,Y,C_ID_7670e569a0,69,3,879,M_ID_00a6ca8a8a,-0.183864,2017-08-31 15:28:01,9,29
466556,Y,C_ID_7670e569a0,116,3,309,M_ID_59764e8cb1,-0.231949,2018-01-12 18:13:41,9,21
466408,Y,C_ID_7670e569a0,116,2,506,M_ID_62826cd868,-0.630452,2017-02-04 12:28:52,9,30
466430,Y,C_ID_7670e569a0,19,2,419,M_ID_85908af96b,-0.621782,2017-03-02 20:32:34,9,1
466480,Y,C_ID_7670e569a0,116,2,299,M_ID_85c4fac890,-0.326165,2017-12-12 13:06:58,9,41
466561,Y,C_ID_7670e569a0,116,2,884,M_ID_89c0061232,-0.445175,2017-07-29 16:55:40,9,27
466537,Y,C_ID_7670e569a0,116,2,45,M_ID_89eb7d619c,-0.371245,2017-02-07 17:16:29,9,18
466406,Y,C_ID_7670e569a0,283,2,45,M_ID_8e28d23e43,-0.372748,2017-03-28 13:50:16,9,18
466453,Y,C_ID_7670e569a0,-1,4,879,M_ID_9139332ccc,-0.476581,2017-07-15 22:11:54,-1,29
466425,Y,C_ID_7670e569a0,283,3,690,M_ID_b1f5106545,0.378578,2017-12-11 16:16:28,9,1


In [145]:
history.loc[history.card_id=='C_ID_06fca19c8e', :].sort_values(['merchant_id','purchase_date']).loc[history.installments!=1,
    history.columns.str.endswith('_id') | history.columns.str.startswith('purchase')|history.columns.str.endswith('installments')
     | history.columns.str.endswith('flag')]

,authorized_flag,card_id,city_id,installments,merchant_category_id,merchant_id,purchase_amount,purchase_date,state_id,subsector_id
21241135,Y,C_ID_06fca19c8e,69,2,511,M_ID_00a6ca8a8a,-0.446378,2017-02-04 14:01:52,9,7
21241127,Y,C_ID_06fca19c8e,69,4,879,M_ID_00a6ca8a8a,1.083983,2017-04-11 14:54:45,9,29
21241129,Y,C_ID_06fca19c8e,69,2,879,M_ID_00a6ca8a8a,-0.589866,2017-06-07 15:52:11,9,29
21241131,Y,C_ID_06fca19c8e,69,5,879,M_ID_00a6ca8a8a,0.934078,2018-01-12 11:09:42,9,29
21241124,Y,C_ID_06fca19c8e,331,3,45,M_ID_0a80430edc,-0.566590,2017-01-16 10:43:23,16,18
21241134,Y,C_ID_06fca19c8e,331,5,422,M_ID_2da30dd32e,-0.331425,2017-05-29 15:43:59,16,27
21241117,Y,C_ID_06fca19c8e,331,3,834,M_ID_4f7d382437,-0.498970,2017-07-26 15:29:58,16,27
21241132,N,C_ID_06fca19c8e,331,999,222,M_ID_629b908f04,1.036739,2017-08-10 13:26:40,16,21
21241119,Y,C_ID_06fca19c8e,331,7,222,M_ID_629b908f04,1.036739,2017-08-10 13:27:31,16,21
21241118,Y,C_ID_06fca19c8e,331,6,222,M_ID_629b908f04,-0.069167,2017-10-05 11:41:48,16,21


#### -1

In [147]:
history.sort_values(['card_id','merchant_id','purchase_date']).loc[history.installments==-1, :]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
26539109,Y,C_ID_0001238066,187,N,-1,NaN,367,M_ID_3f35ef9a50,-1,-0.605012,2018-01-06 12:14:16,1.0,15,16
26539231,Y,C_ID_0001238066,-1,N,-1,NaN,333,M_ID_40d7b120ab,-1,0.244842,2018-01-18 13:58:35,NaN,-1,10
26539141,Y,C_ID_0001238066,160,N,-1,NaN,451,M_ID_970d4d00da,-1,-0.161475,2018-01-23 11:44:30,5.0,21,7
6867524,Y,C_ID_000183fdda,161,N,-1,NaN,422,M_ID_4b99ea1c74,-1,-0.536537,2018-01-12 12:18:39,3.0,3,27
6867550,Y,C_ID_000183fdda,87,N,-1,NaN,369,M_ID_67894eed56,-2,-0.506634,2017-12-28 16:56:20,3.0,11,27
6867520,Y,C_ID_000183fdda,161,N,-1,NaN,68,M_ID_f4e377b35d,-2,-0.091752,2017-12-29 11:31:47,3.0,3,27
6867557,Y,C_ID_000183fdda,161,N,-1,NaN,68,M_ID_f4e377b35d,-1,-0.460833,2018-01-12 10:33:26,3.0,3,27
20951627,Y,C_ID_0002709b5a,103,N,-1,NaN,60,M_ID_5bb84e5819,-4,-0.645779,2017-10-28 00:00:00,2.0,18,13
10374909,Y,C_ID_0003be3c83,279,N,-1,NaN,63,M_ID_d0146c836a,-1,-0.671925,2018-01-06 07:34:03,1.0,9,27
22652959,Y,C_ID_00042d509c,137,N,-1,NaN,544,M_ID_bced6871aa,-2,-0.683195,2017-10-12 03:00:00,3.0,19,29


# category_2

In [93]:
category2_null = history.loc[history.category_2.isnull(), :].sort_values(['card_id', 'merchant_id','purchase_date'], ascending=True)

In [95]:
category2_null.shape

(2652864, 14)

In [96]:
print('category2_null 중에서 unique card_id : ', len(category2_null.card_id.unique()) )
print('category2_null 중에서 unique merchant_id : ', len(category2_null.merchant_id.unique()) )

category2_null 중에서 unique card_id :  167432
category2_null 중에서 unique merchant_id :  15954


In [97]:
category2_null.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
19095862,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-10,-0.728876,2017-04-17 20:20:37,NaN,-1,8
19095805,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-10,-0.728876,2017-04-17 20:29:12,NaN,-1,8
19095883,N,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-8,-0.728876,2017-06-14 14:18:48,NaN,-1,8
19095874,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-8,-0.721363,2017-06-14 14:22:05,NaN,-1,8
19095778,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-7,-0.709342,2017-07-03 15:43:57,NaN,-1,8


In [100]:
history.loc[history.card_id == 'C_ID_00007093c1', : ].sort_values(['purchase_date'])

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
19095896,Y,C_ID_00007093c1,244,N,2,C,705,M_ID_69423b34e4,-12,0.037176,2017-02-14 14:00:43,3.0,2,33
19095775,Y,C_ID_00007093c1,244,N,1,B,705,M_ID_a33355a1b7,-12,-0.467415,2017-02-14 15:47:45,3.0,2,33
19095845,Y,C_ID_00007093c1,244,N,1,B,307,M_ID_9400cf2342,-12,-0.572600,2017-02-16 15:37:58,3.0,2,19
19095866,Y,C_ID_00007093c1,244,N,3,C,705,M_ID_a33355a1b7,-12,-0.331575,2017-02-20 12:19:01,3.0,2,33
19095808,Y,C_ID_00007093c1,244,N,1,B,307,M_ID_9400cf2342,-11,-0.483944,2017-03-03 00:24:15,3.0,2,19
19095905,Y,C_ID_00007093c1,244,N,1,B,307,M_ID_9400cf2342,-11,-0.572600,2017-03-06 18:50:42,3.0,2,19
19095780,Y,C_ID_00007093c1,244,N,1,B,705,M_ID_69423b34e4,-11,-0.401298,2017-03-08 15:05:29,3.0,2,33
19095816,Y,C_ID_00007093c1,244,N,1,B,307,M_ID_9400cf2342,-11,-0.656749,2017-03-08 16:12:35,3.0,2,19
19095833,Y,C_ID_00007093c1,244,N,1,B,705,M_ID_69423b34e4,-11,-0.566590,2017-03-11 15:04:31,3.0,2,33
19095922,Y,C_ID_00007093c1,244,N,1,B,705,M_ID_69423b34e4,-11,-0.677936,2017-03-11 15:10:15,3.0,2,33


In [99]:
history.loc[(history.card_id == 'C_ID_00007093c1') & (history.merchant_id == 'M_ID_57df19bf28')]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
19095778,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-7,-0.709342,2017-07-03 15:43:57,NaN,-1,8
19095785,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,0,-0.721363,2018-02-05 08:56:11,NaN,-1,8
19095790,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-6,-0.721363,2017-08-07 17:46:13,NaN,-1,8
19095797,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-7,-0.721363,2017-07-14 11:51:05,NaN,-1,8
19095802,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-3,-0.721363,2017-11-10 15:34:30,NaN,-1,8
19095803,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-4,-0.721363,2017-10-02 15:37:23,NaN,-1,8
19095805,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-10,-0.728876,2017-04-17 20:29:12,NaN,-1,8
19095809,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-5,-0.709342,2017-09-04 15:07:02,NaN,-1,8
19095810,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-4,-0.709342,2017-10-01 08:05:54,NaN,-1,8
19095812,Y,C_ID_00007093c1,-1,Y,1,B,755,M_ID_57df19bf28,-1,-0.709342,2018-01-30 05:06:15,NaN,-1,8


In [103]:
history.loc[history.merchant_id == 'M_ID_57df19bf28', 'category_2'].isnull().sum()

74153